In [25]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd


# Merge and Reshape Data
    Instead of using Vlookup or Sumif in Excel, how to get a clean and only valid data for data analysis?
    - Joint two dataframe using Key - Database-style DataFrame Merges
    - How to spot the invalid data?  
    - Remove duplicate data
    - Know the average value or the maximum value of certain dataset. This will help to spot an unusual value to validate the data.         
    
Step 1 - sample reinsurnace premium loss data
The Excel file has three tabs, premium/loss/reinsurance.
    - how to display each tab name
    - how to merge data using key "policy number"?


In [6]:
df = pd.ExcelFile('../extra/Premium_Loss.xlsx')
print ('df.sheet_names=%s' % df.sheet_names)

df.sheet_names=['premium', 'loss', 'reinsurance']


In [42]:
fs1 = df.parse("premium")
fs2 = df.parse("loss")
fs3 = df.parse("reinsurance")
df1 = pd.merge(fs1, fs2, on='Policy Number', how='left')
df1.head()

,Policy Number,First Name,Last Name,Policy Effective Date,LOB,Policy Type_x,Premium_x,Coverage,Excess Coverage,Excess Policy,Claim Number,Policy Type_y,Insured First Name,Insured Last Name,Claim Report Date,LAE,Loss,Premium_y
0,10244,Wonda,Hallsworth,2017-08-07,Dentist,OCC,13330,1000000,3900000,92319.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,10509,Patrica,Hartle,2017-02-09,Podiatrist,OCC,5647,1000000,3900000,96730.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,10700,Sabra,Higgenbotham,2017-10-07,PA,OCC,12021,1000000,3900000,90239.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,10880,Kurtis,Dumm,2017-07-08,Phy,CM,232058,1000000,3900000,96289.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,10948,Florencia,Bilyk,2017-06-05,Podiatrist,CM,5132,1000000,3900000,99671.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [43]:
df2=pd.merge(fs1, fs3, on='Policy Number', how='inner')
df2.head()

,Policy Number,First Name,Last Name,Policy Effective Date,LOB_x,Policy Type,Premium_x,Coverage,Excess Coverage,Excess Policy,...,Report Date,Policy Start Date,Policy End Date,Policy Form,Premium_y,Indemnity Reserve,ALAE Reserve,Net ALAE Reserve,Indemnity Payment,ALAE Payment
0,10244,Wonda,Hallsworth,2017-08-07,Dentist,OCC,13330,1000000,3900000,92319.0,...,2018-05-02,2017-08-07,2018-08-07,OCC,13330,50000,20000,NaN,69017,25404
1,10509,Patrica,Hartle,2017-02-09,Podiatrist,OCC,5647,1000000,3900000,96730.0,...,2017-09-16,2017-02-09,2018-02-09,OCC,5647,50000,20000,NaN,78626,91448
2,10700,Sabra,Higgenbotham,2017-10-07,PA,OCC,12021,1000000,3900000,90239.0,...,2018-02-03,2017-10-07,2018-10-07,OCC,12021,50000,20000,NaN,37921,298038
3,10880,Kurtis,Dumm,2017-07-08,Phy,CM,232058,1000000,3900000,96289.0,...,2018-02-03,2017-07-08,2018-07-08,CM,232058,50000,20000,NaN,66591,48555
4,10948,Florencia,Bilyk,2017-06-05,Podiatrist,CM,5132,1000000,3900000,99671.0,...,2018-05-27,2017-06-05,2018-06-05,CM,5132,50000,20000,NaN,75211,256554
